# Problem : Minimize transportation costs

> This analysis is an answer to the question 26 of the first chapter of the book "Introduction to Management Science" from the author Bernard W. Taylor. 

The Easy Drive Car Rental Agency needs 500 new cars in its Nashville operation and 300 new cars in Jacksonville, and it currently has 400 new cars in both Atlanta and Birmingham. It costs \\$30 to move a car from Atlanta to Nashville, \\$70 to move a car from Atlanta to Jacksonville, \\$40 to move a car from Birmingham to Nashville, and \\$60 to move a car from Birmingham to Jacksonville. The agency wants to determine how many cars should be transported from the agencies in Atlanta and Birmingham to the agencies in Nashville and Jacksonville in order to meet demand while minimizing the transport costs. Develop a mathematical model for this problem and use logic to determine a solution.

In [1]:
import pandas as pd

The problem can be modelled considering each possible route as a variable and implementing constraints based on the capacity of each agency.

#### Possible routes with respective prices and variables
Atlanta to Nashville: \\$30 per car, x cars
<br>
Atlanta to Jacksonville: \\$70 per car, y cars
<br>
Birmingham to Nashville: \\$40 per car, w cars
<br>
Birmingham to Jacksonville: \\$60 per car, z cars

#### The cost to be minimized is the sum of the product of each price and each the number of cars moved from a agency to another.
Minimize C = 30*x + 70*y + 40*w + 60*z
<br>
#### And the possible scenarios of cost is restricted by the total number of cars that can be moved from a agency to another:
Subject to:
<br>
x + y = 400
<br>
w + z = 400
<br>
x + w = 500
<br>
y + z = 300

### Method one
In this method, a manual approach was used to find the scenario of transportation that returns the lowest total cost for the combination of each route. No optimization packages were used.

In [2]:
# Here, the variables are put a function of z
from sympy.abc import x, y, w, z #Ao invés de usar o sym.symbols, podemos importar esses já prontos
from sympy import solve

equations = [x + y - 400, w + z - 400, x + w - 500, y + z - 300]           
            
solutions = solve(equations, [x, y, w, z], dict=True)

solutions

[{x: z + 100, y: 300 - z, w: 400 - z}]

In [3]:
# [{x: z + 100, y: 300 - z, w: 400 - z}]

# The restrictions were put as a function of z to reduce the time of calculation. 
#Instead of running thousands of scnearios, only 400 are needed. 400 is the maximum value that z can assume.

combinations = []
limz = 400
i = 0

for z in range(0, limz + 1):
    x = z + 100
    y = 300 - z
    w = 400 - z
    if (x + y == 400) & (w + z == 400) & (x + w == 500) & (y + z == 300) & (x >= 0) & (y >= 0) & (w >= 0):
        combinations.append([i, x, y, w, z, 30*x + 70*y + 40*w + 60*z]) 
        i += 1
    else: pass # pass because we are only interested in the scenarios that meet all restrictions.

In [4]:
# The objective here is to find the scenario with minimal cost.

#First, we find the biggest cost
max_c = 0

for i in range(0,len(combinations)):
    if combinations[i][5] >= max_c:
        max_c = combinations[i][5]
    else: pass
    
# Knowing the biggest cost, we can now compare each scenario to find the one with the lowest cost.
min_c = max_c
n = 0
for i in range(0,len(combinations)):
    if combinations[i][5] <= min_c:
        min_c = combinations[i][5]
        n = i
    else: pass

In [5]:
print('The lowest cost, ${}, is achieved when...'.format(combinations[n][5]))
print('{} cars are moved from Atlanta to Nashville'.format(combinations[n][1]))
print('{} cars are moved from Atlanta to Jacksonville'.format(combinations[n][2]))
print('{} cars are moved from Birmingham to Nashville'.format(combinations[n][3]))
print('{} cars are moved from Birmingham to Jacksonville'.format(combinations[n][4]))

The lowest cost, $34000, is achieved when...
400 cars are moved from Atlanta to Nashville
0 cars are moved from Atlanta to Jacksonville
100 cars are moved from Birmingham to Nashville
300 cars are moved from Birmingham to Jacksonville


### Method two
In this method, PuLP, a optimization package, was used. It saves time of resolution and reduces the time spent writhing code.

In [6]:
from pulp import *
# declare your variables
x = LpVariable("x", 0) # 0<=x
y = LpVariable("y", 0) # 0<=y
w = LpVariable("w", 0) # 0<=w
z = LpVariable("z", 0) # 0<=z

# defines the problem
prob = LpProblem("optimi", LpMinimize)

# defines the constraints
prob += x + y == 400
prob += w + z == 400
prob += x + w == 500
prob += y + z == 300

# defines the objective function to maximize
prob += 30*x + 70*y + 40*w + 60*z

# solve the problem
status = prob.solve()

#print(LpStatus[status])

print('The lowest cost, ${}, is achieved when...'.format(30*value(x) + 70*value(y)+ 40*value(w) + 60*value(z)))
print('{:.0f} cars are moved from Atlanta to Nashville'.format(value(x)))
print('{:.0f} cars are moved from Atlanta to Jacksonville'.format(value(y)))
print('{:.0f} cars are moved from Birmingham to Nashville'.format(value(w)))
print('{:.0f} cars are moved from Birmingham to Jacksonville'.format(value(z)))

The lowest cost, $34000.0, is achieved when...
400 cars are moved from Atlanta to Nashville
0 cars are moved from Atlanta to Jacksonville
100 cars are moved from Birmingham to Nashville
300 cars are moved from Birmingham to Jacksonville
